In [2]:
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import psycopg2
from psycopg2 import sql
from io import BytesIO

# PostgreSQL connection setup
conn = psycopg2.connect(
    host="localhost",
    database="ml_course",
    user="postgres",
    password="admin"
)
cur = conn.cursor()

def create_random_image(size=(28, 28)):
    image = Image.new('L', size, color=255)
    draw = ImageDraw.Draw(image)
    shape = np.random.choice(['rectangle', 'ellipse'])
    x1, y1 = np.random.randint(0, size[0]), np.random.randint(0, size[1])
    x2, y2 = np.random.randint(x1, size[0]), np.random.randint(y1, size[1])
    if shape == 'rectangle':
        draw.rectangle([x1, y1, x2, y2], fill=np.random.randint(0, 200))
    elif shape == 'ellipse':
        draw.ellipse([x1, y1, x2, y2], fill=np.random.randint(0, 200))
    return np.array(image)

def generate_synthetic_customer_data(n_samples=1000):
    np.random.seed(42)
    ages = np.random.randint(18, 80, n_samples)
    genders = np.random.choice(['M', 'F'], n_samples)
    incomes = np.random.normal(50000, 15000, n_samples)
    purchase_frequencies = np.random.choice(['Low', 'Medium', 'High'], n_samples, p=[0.3, 0.5, 0.2])
    return pd.DataFrame({
        'Age': ages,
        'Gender': genders,
        'Income': incomes,
        'PurchaseFrequency': purchase_frequencies
    })

# Drop tables if they exist
cur.execute("""
    DROP TABLE IF  EXISTS synthetic_images
""")

cur.execute("""
    DROP TABLE IF EXISTS synthetic_customers
""")

# Create tables
cur.execute("""
    CREATE TABLE IF NOT EXISTS synthetic_images (
        id SERIAL PRIMARY KEY,
        image BYTEA,
        label INTEGER
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS synthetic_customers (
        id SERIAL PRIMARY KEY,
        age INTEGER,
        gender CHAR(1),
        income FLOAT,
        purchase_frequency TEXT
    )
""")

# Generate and insert synthetic data
num_samples = 1000

for _ in range(num_samples):
    img = create_random_image()
    label = np.random.randint(0, 2)
    img_bytes = BytesIO()
    Image.fromarray(img).save(img_bytes, format='PNG')
    img_bytes = img_bytes.getvalue()
    cur.execute("INSERT INTO synthetic_images (image, label) VALUES (%s, %s)", (psycopg2.Binary(img_bytes), label))

customer_data = generate_synthetic_customer_data(num_samples)
for _, row in customer_data.iterrows():
    cur.execute("""
        INSERT INTO synthetic_customers (age, gender, income, purchase_frequency)
        VALUES (%s, %s, %s, %s)
    """, (row['Age'], row['Gender'], row['Income'], row['PurchaseFrequency']))

conn.commit()
print(f"Generated and stored {num_samples} synthetic images and customer records in PostgreSQL.")

cur.close()
conn.close()

Generated and stored 1000 synthetic images and customer records in PostgreSQL.
